In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.bin.gz

--2024-05-30 14:56:57--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.34.122, 13.226.34.83, 13.226.34.7, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.34.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3934298272 (3.7G) [application/octet-stream]
Saving to: ‘cc.bn.300.bin.gz’

cc.bn.300.bin.gz    100%[===================>]   3.66G  38.2MB/s    in 94s     

2024-05-30 14:58:32 (39.8 MB/s) - ‘cc.bn.300.bin.gz’ saved [3934298272/3934298272]



In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227139 sha256=80195b174bb98ebd182534c9ffe1028938731617cac932595f9dce682b2c1f9c
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!gunzip cc.bn.300.bin.gz

In [ ]:
import fasttext
import fasttext.util
ft = fasttext.load_model('cc.bn.300.bin')

In [ ]:
import pandas as pd
df_train = pd.read_csv("train.csv")[["Headline", "Content", "Label"]]
df_val = pd.read_csv("val.csv")[["Headline", "Content", "Label"]]
df_test = pd.read_csv("test.csv")[["Headline", "Content", "Label"]]

In [ ]:
import fasttext
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np

# Load FastText word vectors
# ft = fasttext.load_model('cc.bn.300.bin')

# Function to get sentence vectors using FastText embeddings THESE ARE FEATURES OF EMBEDDING
def get_sentence_vectors(texts):
    vectors = []
    for text in texts:
        words = text.split()  # Split text into words
        word_vectors = [ft.get_word_vector(word) for word in words]  # Get word vectors for each word
        if word_vectors:  # If there are words in the text
            sentence_vector = np.mean(word_vectors, axis=0)  # Calculate mean of word vectors
        else:  # If no words found
            sentence_vector = np.zeros(ft.get_dimension())  # Use zero vector
        vectors.append(sentence_vector)
    return np.array(vectors)

# Load datasets
# df_train = pd.read_csv("train.csv")[["Headline", "Content", "Label"]]
# df_val = pd.read_csv("val.csv")[["Headline", "Content", "Label"]]
# df_test = pd.read_csv("test.csv")[["Headline", "Content", "Label"]]

# Get sentence vectors for training data
X_train = get_sentence_vectors(df_train['Headline']+" \\\ "+ df_train['Content']) # FEATURES FOR TRAINING | outpit of features.word_emb
y_train = df_train['Label']

# Get sentence vectors for validation data
X_val = get_sentence_vectors(df_val['Headline']+" \\\ "+ df_val['Content'])
y_val = df_val['Label']

# Get sentence vectors for test data
X_test = get_sentence_vectors(df_test['Headline']+" \\\ "+ df_test['Content'])
y_test = df_test['Label']

from sklearn.metrics import f1_score

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'degree': [3, 4, 5],
}

best_f1_score = 0
best_params = None

# Perform Grid Search | HYPERPARAMETER TUNING
for C in param_grid['C']:
    for kernel in param_grid['degree']:
        # Initialize SVM model
        svm_model = SVC(kernel='linear', C=C, cache_size=7000, degree=kernel)

        # Train SVM model
        svm_model.fit(X_train, y_train)

        # Predictions on validation set
        val_predictions = svm_model.predict(X_val)

        # Calculate F1 score
        f1 = f1_score(y_val, val_predictions, average='macro')

        # Update best parameters if current F1 score is higher
        if f1 > best_f1_score:
            best_f1_score = f1
            best_params = {'C': C, 'degree': kernel}

print("Best Parameters:", best_params)

# Retrain the model on the training set using the best parameters
best_model = SVC(kernel='linear', degree=best_params['degree'], C=best_params['C'], cache_size=7000)
best_model.fit(X_train, y_train)

# Predictions on validation set
# val_predictions = best_model.predict(X_val)
# print("Validation Set:")
# print(classification_report(y_val, val_predictions))

# Predictions on test set
test_predictions = best_model.predict(X_test)
print("Test Set:")
print(classification_report(y_test, test_predictions, digits=5))


KeyboardInterrupt: 